## Output Wrong Match

In [1]:
import pandas as pd

In [2]:
thisyear = '2016'
df = pd.read_csv('mc'+ thisyear +'.csv')

In [3]:
error = df[df['meta_movie_name'] == 'error']

In [4]:
noerror = df[df['meta_movie_name'] != 'error']

In [5]:
wrongdate = noerror[~noerror['release_date'].str.contains(thisyear)]

In [6]:
wrongdate = wrongdate.drop_duplicates(subset='rank', keep="first")

In [7]:
df_out = error.append(wrongdate)

In [8]:
df_out.to_csv('mc_error_'+thisyear+'.csv')

## Get MC true matching

In [118]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

In [129]:
thisyear = '2016'
df_error = pd.read_csv('mc_error_'+ thisyear +'v2.csv')
df= pd.read_csv('mc'+ thisyear+'.csv')

In [130]:
df_merge = pd.merge(df, df_error,  how='left', left_on=['rank'], right_on = ['rank'])
df_right = df_merge[df_merge.moviename_y.isnull()]

In [131]:
output_list = []
for i in range(0,len(df_error)):
    moviename = df_error.iloc[i]['moviename']
    rank = df_error.iloc[i]['rank']
    name = df_error.iloc[i]['link']
    print (moviename, '^^^^^', name)
    link1 = 'http://www.metacritic.com/movie/'
    link2 = '/critic-reviews'
    link = link1 + name + link2
    
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    try:
        meta_movie_name = soup.find('div', class_='product_page_title upper pad_top2 pad_btm_half oswald').text.strip()
        num_crit = soup.find('span', class_='based_on').text.strip()
        error = 0
        print ('METANAME @@@@ ', meta_movie_name)
    except:
        error = 1
        print ('ERROR %%% ', name)
        output_list.append([rank, moviename, 'error'])
    
    if error == 0:
        release = soup.find('td', class_='left inset_right2 product_info lighter oswald upper pad_btm1')
        release_date = release.find('span', class_='release_date').text.replace('Release Date:','').strip()
        total_score = soup.find('td', class_='num_wrapper').text.strip()
        try:
            num_crit = soup.find('span', class_='based_on').text.strip()
            num_crit = int(re.search(r'\d+', num_crit).group())
        except:
            num_crit = ''
        try:
            positive = soup.find('div', class_='chart positive')
            positive = positive.find('div', class_='count fr').text.strip()
            mixed = soup.find('div', class_='chart mixed')
            mixed = mixed.find('div', class_='count fr').text.strip()
            negative = soup.find('div', class_='chart negative')
            negative = negative.find('div', class_='count fr').text.strip()
        except:
            positive = ''
            mixed = ''
            negative = ''
            
        critics = soup.find_all('div', class_='review pad_top1 pad_btm1')
        for each in critics:
            iscore = each.find('div', class_='metascore_w').text.strip()
            try:
                source = each.find('span', class_='source').text.strip()
            except:
                source = ''
            try:
                source_link = 'http://www.metacritic.com' + each.find('span', class_='source').a['href']
            except:
                source_link = ''
            try:
                author = each.find('span', class_='author').text.strip()
            except:
                author = ''
            try:
                author_link = 'http://www.metacritic.com' + each.find('span', class_='author').a['href']
            except:
                author_link = ''
            try: 
                date = each.find('span', class_='date').text.strip()
            except:
                date = ''
            try:
                summary = each.find('div', class_='summary').a.text.strip()
            except:
                summary = ''
            output_list.append([rank, moviename, meta_movie_name, release_date, total_score, num_crit, positive, mixed, negative,
                          iscore, source, source_link, author, author_link, date, summary])

I Now Pronounce You Chuck and Larry ^^^^^ i-now-pronounce-you-chuck-larry
METANAME @@@@  I Now Pronounce You Chuck & Larry
Halloween (2007) ^^^^^ halloween-2007
METANAME @@@@  Halloween (2007)
The Water Horse: Legend of the Deep ^^^^^ the-water-horse
METANAME @@@@  The Water Horse
Tyler Perry's Why Did I Get Married? ^^^^^ why-did-i-get-married
METANAME @@@@  Why Did I Get Married?
Aliens Vs. Predator - Requiem ^^^^^ avpr-aliens-vs-predator---requiem
METANAME @@@@  AVPR: Aliens vs Predator - Requiem
The Number 23 ^^^^^ the-number-23
ERROR %%%  the-number-23
Mr. Bean's Holiday ^^^^^ mr-beans-vacation
METANAME @@@@  Mr. Bean's Vacation
Tyler Perry's Daddy's Little Girls ^^^^^ daddys-little-girls
METANAME @@@@  Daddy's Little Girls
The Hills Have Eyes 2 ^^^^^ the-hills-have-eyes-ii
METANAME @@@@  The Hills Have Eyes II
Dragon Wars ^^^^^ dragon-wars-d-war
METANAME @@@@  Dragon Wars: D-War
Pathfinder: Legend of the Ghost Warrior ^^^^^ pathfinder
METANAME @@@@  Pathfinder


In [132]:
new_add = pd.DataFrame(output_list, columns=['rank', 'moviename', 'meta_movie_name', 'release_date', 'total_score', 
                                   'num_crit', 'positive', 'mixed', 'negative', 'iscore', 'source', 'source_link', 'author', 'author_link', 'date', 'summary'])
df_right.drop(['moviename_y', 'link'], axis=1, inplace=True)
df_out = df_right.append(new_add, ignore_index=True)
df_out.to_csv('mc2_'+ thisyear +'.csv')

/Users/santostang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
